In [1]:
from my_packages import *
%matplotlib inline

data_folder = './data/'
pd.options.mode.chained_assignment = None
time_tracking = Time_Tracking()

# prepare data

In [3]:
time_tracking.start_tracking()
dtypes = {'ip':'uint32','app':'uint16','device':'uint16','os':'uint16','channel':'uint16','is_attributed' : 'uint8'}
train_raw = pd.read_csv(data_folder + 'train.csv', dtype = dtypes)
gc.collect()
time_tracking.stop_tracking()

Time used: 3.2  minutes


In [4]:
# train, evaluate split 
data_index = np.arange(len(train_raw))
np.random.seed(123)
np.random.shuffle(data_index)

split_index = int(len(train_raw) * 0.7)
train_df, evaluate_df = train_raw.iloc[data_index[:split_index]], train_raw.iloc[data_index[split_index:]]
gc.collect()

0

# train and cross-validate

In [5]:
time_tracking.start_tracking()

our_model = MyXgbModel()
make_feat_param = {'sample_ratio':1, 'has_target':True, 'do_sample':True}

train_feat = our_model.make_feat_pipeline(train_df, make_feat_param)
gc.collect()

time_tracking.stop_tracking()

Time used: 3.16  minutes


In [6]:
train_param = {'eval_metric':'auc'}
train_x = train_feat.drop(['click_time', 'attributed_time', 'is_attributed', 'os', 'device'], axis = 1)
train_y = train_feat['is_attributed']

train_cv, feat_importance = our_model.train(train_x, train_y, train_param)
gc.collect()
print('train cross-validation auc = {}'.format(round(train_cv,4)))

train cross-validation auc = 0.9371


In [10]:
our_model.dump_model('xgboost_9371.dat')

In [11]:
# our_model = our_model.load_model('xgboost_9371.dat')

In [23]:
# plt.bar(range(len(feat_importance)), feat_importance)
# plt.xticks(range(len(feat_importance)), train_x.columns, rotation='vertical')
# plt.title('feature importance')

# time_tracking.stop_tracking()

# test

In [2]:
# load model
our_model = MyXgbModel()
our_model.load_model('xgboost_9371.dat')

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=nan,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [68]:
# read test raw file
time_tracking.start_tracking()
test_raw = pd.read_csv(data_folder + 'test.csv')
time_tracking.stop_tracking()

Time used: 0.36  minutes


In [69]:
gc.collect()
time_tracking.start_tracking()

hour = []
for t in test_raw['click_time']:
    h = datetime.strptime(t, "%Y-%m-%d %H:%M:%S").hour
    hour.extend([h])
    
time_tracking.stop_tracking()

Time used: 6.0  minutes


In [70]:
time_tracking.start_tracking()

test_raw['click_time_hour'] = hour
test_feat = test_raw[['ip','app', 'channel', 'click_time_hour']]
prediction = our_model.predict(test_feat)

time_tracking.stop_tracking()

Time used: 0.99  minutes


In [71]:
import os


In [72]:
time_tracking.start_tracking()

result = pd.DataFrame({'click_id':test_raw['click_id'], 'is_attributed':prediction})
result.to_csv(data_folder + 'result.csv', index = False)

time_tracking.stop_tracking()

Time used: 1.17  minutes


In [75]:
result.tail()

,click_id,is_attributed
18790464,18790464,0.194290
18790465,18790465,0.066327
18790466,18790467,0.110843
18790467,18790466,0.101524
18790468,18790468,0.033733


In [74]:
os.system('say "your program has finished"')

0